In [13]:
import pandas as pd
import numpy as np
from email import parser
import re 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('all')

from collections import Counter

df = pd.read_csv('./data/labelled_enron/labelled_enron_emails.csv')

# removing root directory crawler values
df = df.iloc[3:]

df

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/ayushgala/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/ayushgala/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/ayushgala/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /Users/ayushgala/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/ayushgala/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nl

,file,message,label
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,1.0
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,1.0
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...,1.0
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...,1.0
7,allen-p/_sent_mail/1004.,Message-ID: <17189699.1075863688308.JavaMail.e...,1.0
...,...,...,...
5026,arnold-j/deleted_items/163.,Message-ID: <11958623.1075852694098.JavaMail.e...,1.0
5027,arnold-j/deleted_items/164.,Message-ID: <31095424.1075852694121.JavaMail.e...,1.0
5028,arnold-j/deleted_items/165.,Message-ID: <10513011.1075852694149.JavaMail.e...,1.0
5029,arnold-j/deleted_items/166.,Message-ID: <28097519.1075852694172.JavaMail.e...,0.0


In [14]:
df.isnull().sum()

file       0
message    0
label      0
dtype: int64

In [15]:
df.dropna()

,file,message,label
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,1.0
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,1.0
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...,1.0
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...,1.0
7,allen-p/_sent_mail/1004.,Message-ID: <17189699.1075863688308.JavaMail.e...,1.0
...,...,...,...
5026,arnold-j/deleted_items/163.,Message-ID: <11958623.1075852694098.JavaMail.e...,1.0
5027,arnold-j/deleted_items/164.,Message-ID: <31095424.1075852694121.JavaMail.e...,1.0
5028,arnold-j/deleted_items/165.,Message-ID: <10513011.1075852694149.JavaMail.e...,1.0
5029,arnold-j/deleted_items/166.,Message-ID: <28097519.1075852694172.JavaMail.e...,0.0


In [16]:
df = df.replace(to_replace='None', value=np.nan).dropna()
df = df.astype(str)

# s = pd.Series(df['EmailContent'])
# s = s.convert_dtypes(convert_string=True)

In [17]:
df.dtypes

file       object
message    object
label      object
dtype: object

In [18]:
df.isna().sum()

file       0
message    0
label      0
dtype: int64

In [19]:
def parse_email(email_content):
    features = {}
    email_parser = parser.Parser()
    
    try:
        email = email_parser.parsestr(email_content)

        # 1. Subject of email
        features['subject'] = email['subject']

        # 2. No. of receivers including CC and BCC
        receivers = []
        for field in ['to', 'cc', 'bcc']:
            if email[field]:
                receivers.extend(re.findall(r'[\w\.-]+@[\w\.-]+', email[field]))
        features['num_receivers'] = len(receivers)

         # 3. Length of email
        body = email.get_payload()
        if email.is_multipart():
            body = ''.join(part.get_payload(decode=True).decode() for part in email.get_payload() if isinstance(part.get_payload(), str))
        features['length_of_email'] = len(body)

        # 4. Email domain name
        from_match = re.search(r'[\w\.-]+@([\w\.-]+)', email['from'])
        features['email_domain'] = from_match.group(1) if from_match else None

        # 6. Sent time
        features['sent_time'] = email['date']

        # 7. Whether the email is forwarded one or not
        features['is_forwarded'] = 1 if (('fwd:' in email['subject'].lower()) or ('fw' in email['subject'].lower())) else 0

        # 8. Sender's email address
        sender_match = re.search(r'([\w\.-]+@[\w\.-]+)', email['from'])
        features['sender_email'] = sender_match.group(1) if sender_match else None

        # # NLP-related semantic features
        stop_words = set(stopwords.words('english'))
        words = word_tokenize(body.lower())
        filtered_words = [word for word in words if word.isalnum() and word not in stop_words]

        # 10. Unique word count (excluding stop words)
        features['email_length'] = len(filtered_words)
    
    except:
        # print(email_content)
        features['subject'] = np.nan
        features['num_receivers'] = np.nan
        features['email_length'] = np.nan
        features['email_domain'] = np.nan
        features['sent_time'] = np.nan
        features['is_forwarded'] = np.nan
        features['sender_email'] = np.nan


    return features

In [20]:
parsed_features = df['message'].apply(parse_email)

# Create new columns in the dataframe with the extracted features
for feature in ['sender_email', 'subject', 'num_receivers', 'email_length', 'email_domain', 'sent_time', 'is_forwarded']:
    df[feature] = parsed_features.apply(lambda x: x[feature])

df.isna().sum()

file             0
message          0
label            0
sender_email     0
subject          0
num_receivers    0
email_length     0
email_domain     0
sent_time        0
is_forwarded     0
dtype: int64

In [21]:
df = df.dropna(axis=0, subset=['email_length', 'email_domain'])
df.isna().sum()

file             0
message          0
label            0
sender_email     0
subject          0
num_receivers    0
email_length     0
email_domain     0
sent_time        0
is_forwarded     0
dtype: int64

In [22]:
df

,file,message,label,sender_email,subject,num_receivers,email_length,email_domain,sent_time,is_forwarded
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,1.0,phillip.allen@enron.com,,1,16,enron.com,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",0
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,1.0,phillip.allen@enron.com,Re: Hello,1,3,enron.com,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",0
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...,1.0,phillip.allen@enron.com,Re: Hello,1,6,enron.com,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",0
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...,1.0,phillip.allen@enron.com,,2,24,enron.com,"Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",0
7,allen-p/_sent_mail/1004.,Message-ID: <17189699.1075863688308.JavaMail.e...,1.0,phillip.allen@enron.com,Re: PRC review - phone calls,1,2,enron.com,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",0
...,...,...,...,...,...,...,...,...,...,...
5026,arnold-j/deleted_items/163.,Message-ID: <11958623.1075852694098.JavaMail.e...,1.0,karen.buckley@enron.com,Telephone Interviews: Trading Track,16,69,enron.com,"Wed, 10 Oct 2001 12:53:27 -0700 (PDT)",0
5027,arnold-j/deleted_items/164.,Message-ID: <31095424.1075852694121.JavaMail.e...,1.0,a..shankman@enron.com,RE:,1,69,enron.com,"Wed, 10 Oct 2001 13:13:05 -0700 (PDT)",0
5028,arnold-j/deleted_items/165.,Message-ID: <10513011.1075852694149.JavaMail.e...,1.0,karen.buckley@enron.com,ENA Trading Track - Interviews October,33,45,enron.com,"Wed, 10 Oct 2001 12:17:23 -0700 (PDT)",0
5029,arnold-j/deleted_items/166.,Message-ID: <28097519.1075852694172.JavaMail.e...,0.0,margaret.allen@enron.com,THANKS!,1,25,enron.com,"Wed, 10 Oct 2001 09:00:39 -0700 (PDT)",0


In [23]:
# Save the updated dataframe to a new CSV file
df.to_csv('./data/parsed/parsed_emails.csv', index=False)